In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

path = "chromedriver-win64/chromedriver-win64/chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()

ticker = "AAPL"
url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(ticker, ticker)

driver = webdriver.Chrome(service=service)
driver.get(url)
driver.implicitly_wait(3)

# Set to keep track of clicked buttons
clicked_buttons = set()

# tmp = button

def click_all_buttons():
    buttons = driver.find_elements(By.XPATH, '//section[@class="container yf-1pgoo1f"]//button')
    clicked_any = False

    for button in buttons:
        button_identifier = button.accessible_name.strip()  # Using button text as an identifier

        # print('button_identifier', button_identifier)

        # If the button has already been clicked, skip it
        if button_identifier in clicked_buttons or button.accessible_name in ["", "Follow", "Quarterly", "Annual", "prev", "next"]:
            continue

        try:
            WebDriverWait(driver, 3).until(EC.element_to_be_clickable(button)).click()
            clicked_buttons.add(button_identifier)  # Mark this button as clicked
            clicked_any = True
            time.sleep(1)  # Adding a small delay to allow new buttons to load
        except Exception as e:
            print(f"Error clicking button: {e}")

    return clicked_any

# Continue to click buttons as long as new ones appear
while True:
    clicked = click_all_buttons()
    if not clicked:
        break  # Exit the loop if no buttons were clicked in the last pass

table = driver.find_element(By.XPATH,  '//div[@class="table yf-1pgoo1f"]')
text_blob = table.text.split("\n")
income_st_dir = {}
last_key = None
for count, row in enumerate(text_blob):
    if count == 0:
        heading = row.split()
        column_count = len(heading[1:])
    else:
        if count%(column_count+1) == 1:
            income_st_dir[row] = []
            last_key = row
        else:
            income_st_dir[last_key].append(row)

driver.quit()


In [4]:
url

'https://finance.yahoo.com/quote/AAPL/financials?p=AAPL'

In [2]:
income_statement_df = pd.DataFrame(income_st_dir).T
income_statement_df.columns = heading[1:]

In [3]:
income_statement_df

,TTM,9/30/2023,9/30/2022,9/30/2021,9/30/2020
Total Revenue,"385,603,000","383,285,000","394,328,000","365,817,000","274,515,000"
Operating Revenue,"385,603,000","383,285,000","394,328,000","365,817,000","274,515,000"
Cost of Revenue,"208,372,000","214,137,000","223,546,000","212,981,000","169,559,000"
Gross Profit,"177,231,000","169,148,000","170,782,000","152,836,000","104,956,000"
Operating Expense,"56,637,000","54,847,000","51,345,000","43,887,000","38,668,000"
Selling General and Administrative,"25,725,000","24,932,000","25,094,000","21,973,000","19,916,000"
Research & Development,"30,912,000","29,915,000","26,251,000","21,914,000","18,752,000"
Operating Income,"120,594,000","114,301,000","119,437,000","108,949,000","66,288,000"
Net Non Operating Interest Income Expense,--,"-183,000","-106,000","198,000","890,000"
Interest Income Non Operating,--,"3,750,000","2,825,000","2,843,000","3,763,000"
